In [10]:
import pickle
import cv2
from sklearn import svm
from scipy.cluster.vq import kmeans, vq
import numpy as np
import pandas as pd
import glob
from skimage.filters import threshold_yen
from skimage.exposure import rescale_intensity
from scipy.stats import skew
from itertools import chain
from skimage.feature import greycomatrix, greycoprops
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler as sc


In [11]:
sift = cv2.SIFT_create()
# load the model from disk
filename = 'frut_recognation_model.sav'
clf_svm_rbf = pickle.load(open(filename, 'rb'))

In [12]:
def get_all_glcm_features(gray_scale_img):
    """
    Given a grayscale image with graylevels from 0 - 255, this function returns the contrast
    and the homogeneity features of the image with the help of GLCM
    """
    # Tip: Make sure you understand the input-output of everything you write, 
    # not doing that results in bugs that make you believe the lab is long
    
    #size of co-occ matrix = number of gray levels
    image_array = np.array(gray_scale_img)
    #print('first pixel= ', image_array[0][0])
    coocurrence_matrix = greycomatrix(image_array, [1], [0])
    contrast = greycoprops(coocurrence_matrix, 'contrast')
    homogeneity = greycoprops(coocurrence_matrix, 'homogeneity')
    #mean = greycoprops(coocurrence_matrix, 'mean')
    energy = greycoprops(coocurrence_matrix, 'energy')
    #entropy = greycoprops(coocurrence_matrix, 'entropy')
    #variance = greycoprops(coocurrence_matrix, 'variance')
    correlation = greycoprops(coocurrence_matrix, 'correlation')
    return contrast, homogeneity, energy, correlation

In [17]:
Test_des_list=[]
Test_hue_arr=[]
Test_data='dataset\\test\\*.jpg'
'''
Apple_training_data='dataset\\Apples\\Test\\*.jpg'
Orange_training_data='dataset\\Oranges\\Test\\*.jpg'
Mango_training_data='dataset\\Mangoes\\Test\\*.jpg'
'''
for file in glob.glob(Test_data):    
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    contrast, homogeneity, energy, correlation= get_all_glcm_features((img_gray * 255).astype(np.uint8))
    kpts, des = sift.detectAndCompute(img_gray,None)
    if len(kpts) < 1:
        no_kpts = np.zeros((1, sift.descriptorSize()), np.float32)
        Test_des_list.append((file, no_kpts))
    else:
        Test_des_list.append((file, des)) 
    h,s,v=cv2.split(img)
    hue_mean=np.mean(h)  #calculate the mean of hue channel of each image
    hue_var=np.var(h) 
    flatten_h = list(np.concatenate(h).flat)
    hue_skewness=skew(flatten_h)
    image_hue_info=[]
    image_hue_info.append(hue_mean)
    image_hue_info.append(hue_var)
    image_hue_info.append(hue_skewness)
    #######################################################################
    image_hue_info.append(contrast[0][0])
    image_hue_info.append(homogeneity[0][0])
    image_hue_info.append(energy[0][0])
    image_hue_info.append(correlation[0][0])
    #########################################################################
    Test_hue_arr.append(image_hue_info) 



    # Stack all the descriptors vertically in a numpy array
descriptors = Test_des_list[0][1]
for file, descriptor in Test_des_list[1:]:
    descriptors = np.vstack((descriptors, descriptor))


#kmeans works only on float, so convert integers to float
descriptors_float = descriptors.astype(float)

k = 60  #k means awal 60 clusters
voc, variance = kmeans(descriptors_float, k, 1)




Test_features = np.zeros((len(Test_hue_arr), k), "float32")
for i in range(len(Test_hue_arr)):
    words, distance = vq(Test_des_list[i][1],voc)
    for w in words:
        Test_features[i][w] += 1



#Test_features_scaled = scale(Test_features.tolist())
#print(Test_features[0],Test_features_scaled[0])

#Test_hue_arr_scaled = scale(Test_hue_arr)
#print(Test_hue_arr[0],Test_hue_arr_scaled[0])

Test_all_features=np.concatenate((Test_hue_arr,Test_features.tolist()),axis=1).tolist()
print(Test_all_features)

[[34.421264962709344, 3412.6617333374807, 1.7043657981202305, 62.783856104908736, 0.6898336105247634, 0.5179499052786559, 0.9896345610805347, 24.0, 16.0, 6.0, 1.0, 10.0, 18.0, 2.0, 47.0, 9.0, 19.0, 19.0, 21.0, 2.0, 17.0, 29.0, 21.0, 32.0, 8.0, 25.0, 17.0, 5.0, 14.0, 9.0, 18.0, 7.0, 34.0, 18.0, 6.0, 37.0, 14.0, 5.0, 16.0, 13.0, 28.0, 25.0, 16.0, 7.0, 54.0, 5.0, 20.0, 8.0, 52.0, 10.0, 16.0, 15.0, 17.0, 13.0, 10.0, 4.0, 6.0, 23.0, 7.0, 6.0, 29.0, 9.0, 37.0, 10.0, 29.0, 16.0, 22.0]]


In [18]:
#predict the fruit type in a new image
#print("LINEAR",clf_linear.predict(Test_all_features))
print("rbf",clf_svm_rbf.predict(Test_all_features))


rbf ['Apple']
